In [ ]:
pip install gender_guesser

In [4]:
import pandas as pd
data=pd.read_csv('../../Dataset_Review/Cleaned/CommentDataset_cleaned_v1.csv')
data.groupby(by=['user_id'])['name'].count().sort_values(ascending=False).head(10)

user_id
 3181651203787409572    449
 234258406790106366     384
 7397842214752822467    298
-2727682506582907011    297
-7932100151854954095    243
-4957063714437462127    223
-1226488663068563871    161
-2209950067799643256    151
 1483740088208321892    145
-7574934947930304997    137
Name: name, dtype: int64

In [ ]:
pip install torch

In [ ]:
pip install transformers

In [ ]:
pip install nltk

In [ ]:
import pandas as pd

import gender_guesser.detector as gender

import re

In [ ]:
reviews = pd.read_csv('CommentDataset_v1.csv')

In [ ]:
d = gender.Detector(case_sensitive=False)

In [ ]:
def solo_un_nombre(name):
    partes = name.split()
    if len(partes) > 1:
        return partes[0]
    else:
        return name

reviews['first_name'] = reviews['name'].apply(solo_un_nombre)

reviews['gender'] = reviews['first_name'].apply(d.get_gender)

In [ ]:
reviews = reviews.drop('first_name', axis=1)

Para nombres que la librería no detecta el género bien pero que a ciencia cierta se sabe cuál es se imputa el valor manualmente.

In [ ]:
reviews.loc[reviews['name'].isin(['Javier', 'Jesus', 'Jesús', 'João']), 'gender'] = 'male'

reviews.loc[reviews['name'].str.startswith('María'), 'gender'] = 'female'
reviews.loc[reviews['name'].isin(['Glória', 'Mary', 'Rocío', 'Lupe',
                                  'Carol', 'Luz', 'Mary Paule', 'Leslie']), 'gender'] = 'female'

Pasaremos ahora a extraer el valor numérico de la columna valoracion.

In [ ]:
def extraer_numeros(texto):
    numeros = re.findall(r'\d+', texto)
    if numeros:
        return int(numeros[0])
    else:
        return None

reviews['rating'] = reviews['valoracion'].apply(extraer_numeros)

In [ ]:
reviews = reviews.drop('valoracion', axis=1)

Pasaremos ahora a hacer el análisis de las fotos de perfil de los usuarios y así poder extraer la información correspondiente al género y a la edad

In [ ]:
pip install deepface

In [ ]:
from deepface import DeepFace

In [ ]:
def analyze_image(image_path):
    try:
        result = DeepFace.analyze(image_path)
        edad = result[0]['age']
        genero = result[0]['dominant_gender']
    except:
        edad = None
        genero = None
    return edad, genero

reviews[['edad', 'genero']] = reviews['imagen'].apply(lambda x: pd.Series(analyze_image(x)))

Ahora pasaremos con el NLP, la extracción de información de cada uno de los apartamentos a partir de los comentarios que los usuarios han hecho.

In [ ]:
import torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment", force_download=True)
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment", force_download=True)

In [ ]:
def calcular_puntaje_sentimiento(df):
    analysis = []
    for comentario in df['comentario']:
        if isinstance(comentario, str):
            # Truncar el comentario si es demasiado largo
            comentario_truncado = comentario[:512]
            tokens = tokenizer.encode(comentario_truncado, return_tensors='pt', max_length=512, truncation=True)
            result = model(tokens)
            score = torch.argmax(result.logits).item() + 1
            analysis.append(score)
        else:
            analysis.append(None)  # Otra opción es asignar algún valor para comentarios que no sean cadenas de texto
    df['analysis'] = analysis

In [ ]:
calcular_puntaje_sentimiento(reviews)

Este dato no se desechará todavía pero se observa que los datos obtenidos son muy similares a los valores de rating que ya se tenían.

In [ ]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Definir una función para limpiar los comentarios
def limpiar_comentario(comentario):
    comentario_sin_puntuacion = re.sub('['+string.punctuation+']', '', comentario)
    comentario_sin_estancia = re.sub(r'\bestancia\b', '', comentario_sin_puntuacion, flags=re.IGNORECASE)
    comentario_sin_dias = re.sub(r'\bdías\b', '', comentario_sin_estancia, flags=re.IGNORECASE)
    return comentario_sin_dias

aparts['comentario_limpio'] = aparts['comentario'].apply(limpiar_comentario)

stopwords_espanol = set(stopwords.words('spanish'))

comentarios_por_apartamento = aparts.groupby('apart_id')['comentario_limpio'].apply(lambda x: ' '.join(x)).reset_index()

textos_por_apartamento = comentarios_por_apartamento['comentario_limpio'].tolist()

palabras_por_apartamento = [[palabra.lower() for palabra in word_tokenize(texto) if palabra.lower() not in stopwords_espanol] for texto in textos_por_apartamento]

palabras_comunes_por_apartamento = [Counter(palabras) for palabras in palabras_por_apartamento]

palabras_x_apartamento = pd.DataFrame({'apart_id': comentarios_por_apartamento['apart_id'],
    'palabras_comunes': palabras_comunes_por_apartamento})

In [ ]:
from collections import Counter

# Palabras clave a buscar en los comentarios
palabras_clave = ['en pareja', 'con pareja', 'en familia', 'con hijos', 'con amigos', 'solo']

# Paso 1: Agrupa los comentarios por apartamento
comentarios_por_apartamento = aparts.groupby('apart_id')['comentario_limpio'].apply(lambda x: ' '.join(x)).reset_index()

# Paso 2: Combina todos los comentarios correspondientes a cada apartamento en un solo texto
textos_por_apartamento = comentarios_por_apartamento['comentario_limpio'].tolist()

# Paso 3: Busca la palabra más común entre las palabras clave en los comentarios de cada apartamento
palabra_mas_comun_por_apartamento = []
for texto in textos_por_apartamento:
    contador_palabras_clave = Counter()
    for palabra_clave in palabras_clave:
        contador_palabras_clave[palabra_clave] = texto.count(palabra_clave)
    palabra_mas_comun = contador_palabras_clave.most_common(1)
    palabra_mas_comun_por_apartamento.append(palabra_mas_comun[0][0] if palabra_mas_comun else None)

# Crear un DataFrame con los resultados
df_palabra_mas_comun = pd.DataFrame(palabra_mas_comun_por_apartamento, columns=['Palabra Más Común'])

# Agregar el id del apartamento como índice
df_palabra_mas_comun['apart_id'] = comentarios_por_apartamento['apart_id']
df_palabra_mas_comun = df_palabra_mas_comun.set_index('apart_id')

In [ ]:
reviews.to_csv('CommentDataset_final.csv')

In [ ]:
df_palabra_mas_comun.to_csv('CompaniaApart.csv')

In [ ]:
palabras_x_apartamento.to_csv('PalabrasApart.csv')